## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hanninga (`np.hanning`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:

        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate
        
lena = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

In [ ]:
def twoD_fourier(img):
    img = np.float32(img)
    
    dtf = cv2.dft(img, flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dtf, [0, 1])
    
    magnitude, phase = cv2.cartToPolar(dft_shift[:,:,0], dft_shift[:,:,1])

    return dft_shift

def inverse(img):
    img_shifted = np.fft.ifftshift(img, [0,1])
    ifft = cv2.idft(img_shifted, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    img_ifft = cv2.magnitude(ifft[:, :, 0], ifft[:, :, 1])
    img_inv = np.round(img_ifft).astype('uint8')
    return img_inv

def show_diff(img1, img2):
    plt.figure(figsize=(12, 12))
    plt.subplot(131)
    plt.imshow(img1, 'gray')
    plt.title('Image 1')
    plt.axis('off')
    plt.subplot(132)
    plt.imshow(img2, 'gray')
    plt.title('Image 2')
    plt.axis('off')
    plt.subplot(133)
    plt.imshow(np.abs(img1 - img2), 'gray')
    plt.title('Difference')
    plt.axis('off')
    plt.show()

In [ ]:
# kroki 1-3
filter_size = 21

hanning_window = np.hamming(filter_size)

hanning_2d = np.outer(hanning_window, hanning_window)

In [ ]:
# krok 4
FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(filter_size))
FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, filter_size]))
FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(filter_size))
FSpaceColsM = np.outer(np.ones([1, filter_size]), FSpaceCols)   

FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

FilterF = FreqR <= 0.2

In [ ]:
# krok 5
FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
FilterFidft = cv2.idft(np.float32(FilterFRot3), flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

In [ ]:
# krok 6
ProperlyFilter = hanning_2d * FilterFI

In [ ]:
# krok 7
border = cv2.copyMakeBorder(ProperlyFilter, 0, 491, 0, 491, cv2.BORDER_CONSTANT, value=0)

border_fft = twoD_fourier(border)
magnitude = cv2.magnitude(border_fft[:,:,0], border_fft[:,:,1])

magnitudeF3 = np.repeat(magnitude[:, :, np.newaxis], 2, axis=2)

abs_border_fft = np.abs(border_fft)

In [ ]:
# krok 8
lena_fft = twoD_fourier(lena)

l_size = lena.shape[0]  # obraz LENA jest kwadratowy
FSpaceRows_l = 2*np.fft.fftshift(np.fft.fftfreq(l_size))
FSpaceRowsM_l = np.outer(FSpaceRows_l, np.ones([1, l_size]))
FSpaceCols_l = 2*np.fft.fftshift(np.fft.fftfreq(l_size))
FSpaceColsM_l = np.outer(np.ones([1, l_size]), FSpaceCols_l)  

figFilter = plt.figure()
axsFilter = figFilter.add_subplot(projection='3d')
axsFilter.plot_surface(FSpaceRowsM_l, FSpaceColsM_l, abs_border_fft[:,:,0], 
                       rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)

axsFilter.set_title("Wizualizacja filtra")
plt.show()

In [ ]:
# wyświetlenie wyników działania filtra dla obrazu LENA
filter_hanning = magnitudeF3 * lena_fft
lena_hanning = inverse(filter_hanning)

show_diff(lena, lena_hanning)

In [ ]:
# wyświetlenie wyników działania  ZWYKŁEGO filtra  bez mnożenia przez okno Hanninga
FreqR = np.sqrt(np.square(FSpaceRowsM_l) + np.square(FSpaceColsM_l))
FilterF = FreqR <= 0.2

filter_without_hanning = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)
filter_lena = filter_without_hanning * lena_fft
lena_no_hanning = inverse(filter_lena)

show_diff(lena, lena_no_hanning)

### Porównując wyniki działania powyższych dwóch flitrów (dla tej samej częstotliwości) widać znaczną różnicę, na korzyść tego wykorzystującego metodę okien. Nie występują artefakty, tylko lekkie rozmycie obrazu.